# Repertoire

In [ ]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")

Mounted at /gdrive


In [ ]:
REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

# MODULES

In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
%%capture
pip install -r requirements.txt

In [ ]:
import re
import pandas as pd
from datetime import datetime


import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


# FUNCTIONS

In [ ]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Discogs data

## Load 

In [ ]:
baseline = pd.read_pickle(REP_INTERMED+ "baseline.sav")[['master_id','artist',	'title',	'text_CLEAN']]

## Vectorizer :  Keep TF-IDF strategy 

**La cellule ci-dessous exporte le vectorizer et X_TRAIN!!!! L' exécution n'est pas nécessaire à partir de la deuxième exécution**

In [ ]:
start_time = datetime.now()


create_vectorizer_and_vectorized_db(baseline, analyzer = 'char',ngram_range=(2,2))

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

# Duration: 02 min:15s

**** creation du vectorizer


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


nb features: 3570
**** sauvegarde des fichiers en pickle pour la prochaine fois
Duration: 0:03:44.605118


In [ ]:
analyzer = 'char'

ngram_range = (2,2)
X_train, vectorizer = reload_data_and_vectorizer(analyzer = analyzer, ngram_range=ngram_range)

# Query Data

In [ ]:
df_no_match = pd.read_pickle(REP_INTERMED +'base_cdandlp.sav').reset_index(drop=True)

df_no_match['text'] = df_no_match['artiste'] +  ' ' + df_no_match['titre']

df_sample = df_no_match.sample(10000, random_state=20).reset_index(drop=True)

del df_no_match


# KNN unsupervised (sklearn api)

## Training with Manahattan

In [ ]:
from functools import partial

baseline['index'] = baseline.index

start_time = datetime.now()

func = partial(knn_scratch, vectorizer = vectorizer, X_train = X_train, baseline = baseline.copy(), sample =df_sample, nb_top=20)

answer = list(map(func,      [[i, df_sample.loc[i,'text']] for  i in range(len(df_sample))]))

with open(REP_INTERMED + 'results_sim3.pkl', 'wb') as f:
  pickle.dump(answer, f) 

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

# Customiz Output

In [ ]:
with open(REP_INTERMED + 'results_sim3.pkl', 'rb') as file:
      result = pickle.load(file)


pd.concat(result).to_excel(REP_INTERMED + 'full_cosine_knn.xlsx', index=False)